<a href="https://colab.research.google.com/github/SASambath69/notebooks/blob/main/Analyse_des_Messages_d'un_Groupe_WhatsApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objectifs

Le présent notebook a été réalisé pour effectuer une analyse des discussions d'un groupe WhatsApp. Les informations concernant les expéditeurs seront anonymisées. Les objectifs seront les suivants :

*    Combien de messages ont été envoyés dans ce groupe au total par année ? Puis par mois ?
*    Qui sont les contributeurs les plus actifs ?
*    A quel moment de la journée et quel jour de la semaine il y a le plus de messages ?
*    Quels sont les mots les plus utilisés ?
*    Quels sont les emojis les plus présents ?

# Cheminement

1.    Récupération du fichier de sauvegarde de la discussion (.txt)
2.    Retraitement du fichier et création de DataFrames
3.    Sauvegarde des DataFrames dans un fichier Excel
4.    Mise en place d'un rapport Power BI présentant les informations requises

# Import des Librairies

Petites précisions sur quelques librairies utilisées :

*    nltk : pour l'analyse des mots contenus dans les messages
*    string : servira pour retirer les ponctuations lors de l'analyse des mots
*    emoji : pour l'analyse des emojis les plus utilsés
*    InteractiveShell : pour afficher tous les outputs d'un bloc de code, et ne pas se limiter au dernier output
*    drive : pour me connecter à mon Google Drive et récupérer le fichier txt correspondant à la sauvegarde de la discussion WhatsApp
*    google.colab files : pour télécharger le fichier Excel contenant les DataFrames qui serviront de base à la mise en place du Dashboard

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('popular')
import string
!pip install emoji
import emoji
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import datetime
from google.colab import files

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Import et Retraitement du fichier contenant les discussions

Par défaut, les discussions vont être retranscris dans un DataFrame à 1 colonne. Les retraitements et travaux seront les suivants :

1.    Identification des lignes qui correspondent à des suites de messages. En effet, selon la longueur des messages, ceux-ci peuvent apparaître sur plusieurs lignes du DataFrame. L'objectif est ainsi de les repérer selon différents moyens (longueur du texte inférieur à un certain nombre ou absence de date dans les 1ers caractères du message) puis de les replacer à la suite des messages dont ils correspondent
2.    Création d'une colonne contenant la date et heure de l'envoi du message
3.    Création d'une colonne contenant le nom de l'expéditeur et anonymisation
4.    Création de 2 colonnes correspondant au jour de la semaine (de 1 à 7) et à l'heure de la journée (de 0 à 23), puis pivot table pour afficher le nombre de messages envoyés par heure de la jour et par jour de la semaine
5.    Création d'une colonne contenant le message
6.    Identification des mots les plus utilisés (ajout de certains mots, expressions, caractères à la liste des stopwords). Pour les messages contenant des médias, ceux-ci apparaissent avec la mention "<Médias omis>". Je décide de ne pas en tenir compte
7.    Identification des emojis les plus utilisés

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Discussion WhatsApp avec Le Verty Link Brignairot.txt', delimiter = "\t", header = None, names = ['text'])

# ------------------------------ Identification des lignes correspondant à des suites de message et replacement de ces derniers à la suite des messages auxquels ils appartiennent

df['follow_message'] = df['text'].apply(lambda x: True if len(x) < 10 else (True if x[2] != '/' or x[5] != '/' else False))
df['text_def'] = df['text']
for i in range(len(df['text'])):
  if i != len(df['text'])-1 and df['follow_message'][i] == False and df['follow_message'][i+1] == False:
    pass
  elif df['follow_message'][i] == True:
    start = i
    while df['follow_message'][start] == True:
      df['text_def'][i-1] = df['text_def'][i-1] + ' ' + df['text_def'][start]
      start += 1

df = df.loc[df['follow_message'] == False].drop(columns=['text','follow_message'])

# ------------------------------ Extraction des dates et heures

df[['datetime_str','no_date']] = df["text_def"].str.split(' - ', 1, expand=True)
df['datetime_str'] = df['datetime_str'].str.replace(' à', '')
df['datetime_str'] = pd.to_datetime(df['datetime_str'])

# ------------------------------ Extraction des expéditeurs

df = df.loc[df['no_date'].str.contains(':')==True]
df[['sender','text_message']] = df['no_date'].str.split(': ', 1, expand=True)

# ------------------------------ Anonymisation

senders = list(df['sender'].unique())
anonyme = []
for i in range(len(df['sender'].unique())):
  anonyme.append('sender_{}'.format(i+1))
df['sender'] = df['sender'].replace(senders, anonyme)

# ------------------------------ Stopwords

stop_words = list(stopwords.words('french'))
stop_words.extend(['a','’','je','ça','ca','c','j','et','...','va',"c'est",'vais','fait',"''",'``',"j'ai",'^^'])

# ------------------------------ Mots les plus utilisés par expéditeur (tout le monde)

df_by_sender = pd.DataFrame()

# Certains messages correspondent en réalité à des médias et sont inscrits "<Médias omis>". On n'en tient pas compte pour la suite

for i in range(len(df['sender'].unique())):
  sender = df['sender'].value_counts().index[i]
  df_sender = df.loc[df['sender'] == sender].loc[df['text_message'].str.lower().str.contains('médias omis')==False]
  all_words_sender = ''
  for j in df_sender['text_message']:
    all_words_sender = all_words_sender + ' ' + j
  df_words_sender = pd.DataFrame({'sender' : sender, 'words' : word_tokenize(all_words_sender)})
  df_words_sender['no_stop_words'] = df_words_sender['words'].apply(lambda x: x.lower() if x.lower() not in stop_words and x.lower() not in list(string.punctuation) and x not in emoji.UNICODE_EMOJI['en'] else np.NaN)
  df_words_sender_top = pd.DataFrame({'sender' : sender, 'words' : list(df_words_sender['no_stop_words'].value_counts().head(20).index), 'count' : list(df_words_sender['no_stop_words'].value_counts().head(20).values)})

  df_by_sender = pd.concat([df_by_sender, df_words_sender_top], axis = 0)

del df_sender
del df_words_sender
del df_words_sender_top

# ------------------------------ Emojis les plus utilisés par expéditeur (tout le monde)

df_emo_by_sender = pd.DataFrame()

# Certains messages correspondent en réalité à des médias et sont inscrits "<Médias omis>". On n'en tient pas compte pour la suite

for i in range(len(df['sender'].unique())):
  sender = df['sender'].value_counts().index[i]
  df_sender = df.loc[df['sender'] == sender].loc[df['text_message'].str.lower().str.contains('médias omis')==False]
  all_words_sender = ''
  for j in df_sender['text_message']:
    all_words_sender = all_words_sender + ' ' + j
  df_words_sender = pd.DataFrame({'sender' : sender, 'words' : word_tokenize(all_words_sender)})
  df_words_sender['emoji'] = df_words_sender['words'].apply(lambda x: x if x in emoji.UNICODE_EMOJI['en'] else np.NaN)
  df_words_sender_top = pd.DataFrame({'sender' : sender, 'words' : list(df_words_sender['emoji'].value_counts().head(20).index), 'count' : list(df_words_sender['emoji'].value_counts().head(20).values)})

  df_emo_by_sender = pd.concat([df_emo_by_sender, df_words_sender_top], axis = 0)

del df_sender
del df_words_sender
del df_words_sender_top

# Sauvegarde des DataFrames dans un fichier Excel et téléchargement de ce fichier

In [ ]:
with pd.ExcelWriter('whatsapp_{}.xlsx'.format(str(datetime.date.today()))) as writer:
  df[['datetime_str','sender']].to_excel(writer, sheet_name='All')
  df_by_sender.to_excel(writer, sheet_name='Words')
  df_emo_by_sender.to_excel(writer, sheet_name='Emoji')

files.download('whatsapp_{}.xlsx'.format(str(datetime.date.today())))

# Rapport Power BI

[Lien vers le rapport](https://app.powerbi.com/view?r=eyJrIjoiZjUyN2Y4YjUtYTZiMi00YjZjLWE1NjItMzZlZWNiNjZkM2MyIiwidCI6IjE0NTJmNzE3LTQ5MTItNDE1Yi1hZjg1LWQ3Njc5YWM0MWQwNiJ9)